In [62]:
class Movie:
    def __init__(self, title, year, rating, thumbnail_url, movie_url, actors):
        self.title = title
        self.rating = rating
        self.thumbnail_url = thumbnail_url
        self.actors = actors
class Actor:
    def __init__(self, name, url):
      self.name = name
      self.url = url

In [72]:
import requests
import json

from pymongo import MongoClient
from bs4 import BeautifulSoup

client = MongoClient('_____')
actor_db = client.movieAppData.actors
movie_db = client.movieAppData.movies


URL = 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

result = soup.find("tbody", {"class":"lister-list"}).findAll('tr')
count = 0
for tag in result:
    if(count <3):
        title = tag.find("td",{"class":"titleColumn"}).a.text
        movie_year = "" if tag.find("td",{"class":"titleColumn"}).span.text == None else tag.find("td",{"class":"titleColumn"}).span.text
        rating = "Nil" if tag.find("td",{"class":"ratingColumn imdbRating"}).strong == None else tag.find("td",{"class":"ratingColumn imdbRating"}).strong['title']
        thumbnail_url = tag.find("td",{"class":"posterColumn"}).img['src']

        movie_url = "https://www.imdb.com" + tag.find("td",{"class":"titleColumn"}).a['href']
        # Scrape and get the actors of the current link
        movie_page = requests.get(movie_url)
        movie_soup = BeautifulSoup(movie_page.content, 'html.parser')
        actors = [i.text for i in movie_soup.findAll("div",{"class":"credit_summary_item"})[2].findAll("a")]
        actors = actors[0:len(actors)-1] # Remove the last index because it always is "See the full cast"
        actor_urls = ["https://www.imdb.com" + i["href"] for i in movie_soup.findAll("div",{"class":"credit_summary_item"})[2].findAll("a")]
        actors_list = []
        for i in range(len(actors)):
            actor = Actor(actors[i], actor_urls[i])
            # Add into mongo
            actor_db.insert_one(actor.__dict__)

        # Create the movie object 
        movie = Movie(title, movie_year, rating, thumbnail_url, movie_url, actors)
        # Add into mongo
        movie_db.insert_one(movie.__dict__)


    
    count += 1
    print(count)
#     print(json.dumps(m.__dict__))
#     print(json.dumps(actor.__dict__))
    
    



<ipython-input-72-dee38bca5ac7>:37: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  actor_db.insert(actor.__dict__)
<ipython-input-72-dee38bca5ac7>:42: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  movie_db.insert(movie.__dict__)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
